### Set Up

In [ ]:
import ealib
import logging
import pandas as pd
from collections import Counter
from typing import List
import random
from typing import Tuple
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np

# Required to identify with EDGAR API
req_header = {"User-Agent": "roberto.brera.24@outlook.com"}

# Select rate of requests (< 10)
mrps = 8

# Select desired logging level
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

# Stores information about all tickers currently available on SEC database
tickers_df = ealib.get_tickers_df(req_header)

### Utility Scripts

#### Select Ticker

In [ ]:
query_substr =  "micro"
query_ticker = ealib.find_title_substring(tickers_df, query_substr).iloc[0]
comp_str = f'{query_ticker["title"]} ({query_ticker["ticker"]}, Index {query_ticker.name})'

# Retrieve main dictionaries required for this ticker
comp_mtd = ealib.get_response_dict(ealib.metadata_url(query_ticker["cik_str"]), req_header, mrps)
if comp_mtd is None:
    logging.warning(f'Failed to retrieve company metadata for ticker {query_ticker["ticker"]}')
comp_facts = ealib.get_response_dict(ealib.companyfacts_url(query_ticker["cik_str"]), req_header, mrps)
if comp_facts is None:
    logging.warning(f'Failed to retrieve company facts for ticker {query_ticker["ticker"]}')

query_ticker

#### Download Select Filings

In [ ]:
ealib.download_company_filings(
    req_header, mrps, 
    comp_dir="Adecoagro Tests", 
    select_filings=ealib.filter_filings(
        pd.DataFrame.from_dict(comp_mtd["filings"]["recent"]), 
        "filingDate", "form", 

        query_forms=[""], 
        max_days=1080,
        min_days=0
        
        ),
    ticker=query_ticker, 
    write_txt=False, 
    write_pdf=True
)

#### Obtaining a company fact using yf

In [ ]:
# Start by making yf ticker obejct
curr_yticker = yf.Ticker(query_ticker["ticker"])

In [ ]:
# Search for particular info keys using search functions
ealib.find_dict_key_substr(curr_yticker.info, ["currency"])
ealib.find_dict_key_substr(curr_yticker.info, ["cap"])
ealib.find_keys_containing_all_substrs(curr_yticker.info, ["cash", "operating"])

In [ ]:
# Use encapsulated function to search safely for the company fact, returning None if not found
ealib.yf_info(query_ticker["ticker"], "marketCap")
ealib.yf_info(query_ticker["ticker"], "currency")
ealib.yf_info(query_ticker["ticker"], "operatingCashflow")

In [ ]:
# Generate info series for a df of tickers (can be directly stored on source df)
marketCap_series = tickers_df[:100]["ticker"].apply(lambda x: ealib.yf_info(x, "marketCap"))
marketCap_series

#### Saved Refined Query Strings for Company Facts

In [ ]:
shares_outstanding_query_str = ["NumberOfSharesOutstanding", "EntityCommonStockSharesOutstanding", "CommonStockSharesOutstanding"]
ocf_query_str = ["NetCashProvidedByUsedInOperatingActivities", "CashFlowsFromUsedInOperatingActivities"]
# To use with sufficient = True

#### Interactive Company Fact Search

In [ ]:
print_all_comp_fact_df = False
display_short_comp_fact_df = False

# If you do not yet have query str -> Broad search
query_fact_substr = ["cash", "operat"]
sufficient = False

# If you have query str -> Refined search 
"""
query_fact_substr = ["NumberOfSharesOutstanding", "EntityCommonStockSharesOutstanding", "CommonStockSharesOutstanding"] # Refined list
sufficient = True
"""

# Navigate company facts dictionary and find all matches to query company fact
res = ealib.comp_facts_df(
    comp_facts,
    query_fact_substr, 
    sufficient,
)
for units, as_key, match_fact, comp_fact_df in res:
    logging.info(f'Found company fact matching {query_fact_substr}: as_key = {as_key}, match_fact = {match_fact}, units = {units}')
    logging.info(f'\n\t{comp_fact_df}') if print_all_comp_fact_df else None
if res: # If only one company fact required, select closest match by choosing shortest matching string
    res_units, res_as_key, res_match_fact, res_comp_fact_df = min(res, key=lambda x: len(x[2]))
    logging.info(f'Shortest company fact match:\n\t res_as_key = {res_as_key}, res_match_fact = {res_match_fact}, res_units = {res_units}')
    display(res_comp_fact_df) if display_short_comp_fact_df else None

#### Analyze presence of selected company fact across tickers_df

In [ ]:
# Search company fact with refined query string 
query_fact_substr = ["NumberOfSharesOutstanding", "EntityCommonStockSharesOutstanding", "CommonStockSharesOutstanding"] 
sufficient = True
print_short_comp_fact_df = False
num_tests = 10

len_counter = Counter()
for _ in range(num_tests):
    random_number = random.randint(1, 10000)
    curr_ticker = tickers_df.iloc[random_number]
    comp_str = f'ticker number: {random_number}; ticker: {curr_ticker["ticker"]}; company title: {curr_ticker["title"]}'

    # Request comp facts dictionary
    comp_facts = ealib.get_response_dict(ealib.companyfacts_url(curr_ticker["cik_str"]), req_header, mrps)
    if comp_facts is None:
            logging.warning(f'Failed request when attempting to retrieve company facts for {comp_str}')
            len_counter['failed reqs'] += 1

    # Extract desired company fact
    res = ealib.comp_facts_df(
        comp_facts,
        query_fact_substr, 
        sufficient,
    )
    # Record the number of matches
    len_counter[f'{len(res)}'] += 1

    for units, as_key, match_fact, comp_fact_df in res:
        logging.info(f'Found company fact matching {query_fact_substr}: as_key = {as_key}, match_fact = {match_fact}, units = {units} with {comp_str}')
    if res: # Now extract tuple with shortest match_fact
        res_units, res_as_key, res_match_fact, res_comp_fact_df = min(res, key=lambda x: len(x[2]))
        logging.info(f'Shortest match fact tuple selected:\n\t res_as_key = {res_as_key}, res_match_fact = {res_match_fact}, res_units = {res_units} for test with {comp_str}')
        logging.info(res_comp_fact_df) if print_short_comp_fact_df else None
        
len_counter


#### Graphing a Company Fact Over Time

##### Select Filings for Graph's Values

In [ ]:
"""
    Run Interactive Company Fact Search first, establishing following variables
    res_units, res_as_key, res_match_fact, res_comp_fact_df
"""

# First check the filings on which graph will be based
filing_date_col = "end"
y_axis_col = "val"
max_days = 360

select_filings_df = ealib.filter_filings(res_comp_fact_df, filing_date_col=filing_date_col, form_col="form", query_forms=[""], max_days=max_days)
select_filings_df

##### Plot Graph

In [ ]:
plt.figure(figsize=(10, 5))  # Set the figure size (optional)
plt.plot( select_filings_df[filing_date_col], select_filings_df[y_axis_col], marker='o')  # Line plot with markers
plt.title(f'{res_match_fact} for {comp_str}')  # Adding a title to the graph
plt.xlabel('Filing end date')  # Label for the x-axis
plt.ylabel(f'Reported Number of {res_match_fact}')  # Label for the y-axis
plt.grid(True)  # Enable grid for easier readability
plt.xticks(rotation=45)
plt.show()  # Display the plot

#### Calculating cash burn rate

In [ ]:
""" 
Calculates average cash burn daily rate, according to all SEC filing data available over the specified period
"""
min_days = 0
max_days = 360
query_forms = [""]
filing_date_col = "filed"

res = ealib.comp_facts_df(
    comp_facts,
    ["NetCashProvidedByUsedInOperatingActivities", "CashFlowsFromUsedInOperatingActivities"], 
    True
)
if res:
    res_units, res_as_key, res_match_fact, res_comp_fact_df = min(res, key=lambda x: len(x[2]))
    ocf_df_filt = ealib.filter_filings(res_comp_fact_df, filing_date_col=filing_date_col, form_col="form", query_forms=query_forms, max_days=max_days, min_days=min_days)
    display(f'Average daily OCF burn rate {ealib.ocf_average_daily_burn_rate(ocf_df_filt)}, with units {res_units} and company fact {res_match_fact}, under accounting standards {res_as_key}')


#### Converting between currencies

In [ ]:
from_currency = "USD"
to_currency = "USD"
forex_ticker =  f"{from_currency}{to_currency}=X"

ealib.yf_info(forex_ticker, "previousClose")